In [1]:
from aicsimageio import AICSImage
import numpy as np
from skimage import io
import tifffile

In [2]:

input_czi_file_path = '/Users/apple/Desktop/Akamatsu_Lab/Data/TwoSlide_Left_JFXRapidWithWash-05_processed.czi'

# Load the CZI file
img = AICSImage(input_czi_file_path)


In [29]:
# Check which reader is being used
print(type(img.reader))

<class 'aicsimageio.readers.czi_reader.CziReader'>


In [3]:
img.dask_data

dask.array<transpose, shape=(130, 3, 116, 2052, 340), dtype=uint16, chunksize=(1, 1, 116, 2052, 340), chunktype=numpy.ndarray>

In [4]:
img.xarray_dask_data

<xarray.DataArray 'transpose-dcf513a04319699e1ab32fe3d600cf73' (T: 130, C: 3,
                                                                Z: 116,
                                                                Y: 2052, X: 340)> Size: 63GB
dask.array<transpose, shape=(130, 3, 116, 2052, 340), dtype=uint16, chunksize=(1, 1, 116, 2052, 340), chunktype=numpy.ndarray>
Coordinates:
  * C        (C) <U6 72B '640 nm' '488 nm' '561 nm'
  * Z        (Z) float64 928B 0.0 0.145 0.29 0.435 ... 16.24 16.38 16.53 16.67
  * Y        (Y) float64 16kB 0.0 0.145 0.29 0.435 ... 296.9 297.1 297.2 297.4
  * X        (X) float64 3kB 0.0 0.145 0.29 0.435 ... 48.72 48.86 49.01 49.15
Dimensions without coordinates: T
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x7f9d6eb739c0>

In [5]:
img.dims

<Dimensions [T: 130, C: 3, Z: 116, Y: 2052, X: 340]>

In [6]:
img.dims.order

'TCZYX'

In [7]:
img.dims.X

340

In [28]:
lazy_t0 = img.get_image_dask_data("CZYX", T=1,C=2)
lazy_t0.shape
t0 = lazy_t0.compute() 
t0.shape

(1, 114, 2048, 337)

In [20]:
# Pull only a specific chunk in-memory
lazy_t0 = img.get_image_dask_data("ZYX", T=20, C=0)  # returns out-of-memory 4D dask array
lazy_t0.shape
t0 = lazy_t0.compute()  # returns in-memory 4D numpy array

In [21]:
t0.shape

(114, 2048, 337)

In [22]:
t0_0 = t0
t0_0.shape

(114, 2048, 337)

In [23]:
output_tiff_file_path = '/Users/apple/Desktop/Akamatsu_Lab/Data/output_image.tif'

In [24]:
# Save the array to a TIFF file
io.imsave(output_tiff_file_path, t0)

/var/folders/wk/40y5g1gd3y73hktn1mf1hbwc0000gn/T/ipykernel_11212/468939077.py:2: UserWarning: /Users/apple/Desktop/Akamatsu_Lab/Data/output_image.tif is a low contrast image
  io.imsave(output_tiff_file_path, t0)


In [25]:
# Save the array as OME-TIFF
with tifffile.TiffWriter(output_tiff_file_path, bigtiff=False) as tif:
    tif.write(t0, photometric='minisblack', metadata={'axes': 'ZYX'})

In [ ]:
input_filename_czi = '/Users/apple/Desktop/Akamatsu_Lab/Data/TwoSlide_Left_JFXRapidWithWash-05_processed.czi'

# Specify the directory where you want to save the pickle files
save_dir = '/Users/apple/Desktop/Akamatsu_Lab/pyLattice_tutorials/DataFrames/Channel_3_pkl_full'

# csv saving directory 
#csv_save_dir = '../DataFrames/Channel_3_csv'

In [ ]:
import time

# Load the CZI file
img = AICSImage(input_filename_czi)

#get the number of frames for our original data for automated analysis for all frames 
frames = img.dims.T
start_frame = 0
end_frame = 1

start_time = time.time()

for frame in range(start_frame,end_frame):
    
    print('frame number is', frame)
    lazy_single_frame_input = img.get_image_dask_data("ZYX", T=frame, C=1)
    
    single_frame_input = lazy_single_frame_input.compute()
    print(single_frame_input.shape)
    single_frame_input = np.transpose(single_frame_input,axes =(0,2,1))
    print(single_frame_input.shape)

    #define threshold: a value(intensity) for the pixel below which all values would be considered noise and dropped 
    #define min_distance: min_distance/2 is the radius within which we will keep the peak with max value/intensity or 
    #if two peaks have the same value they will be kept 
    
    maximas = peak_local_max_3d(single_frame_input,min_distance=10,threshold=180)
    print('local_maximas detected are', maximas.shape[0])


    #give the expected std dev/radius of our particles for x,y,z 
    sigmaExpected_x__pixels = 2
    sigmaExpected_y__pixels = 2
    sigmaExpected_z__pixels = 4

    sigmas_guesses = []
    for i in range(len(maximas)):
        sigmas_guesses.append([sigmaExpected_z__pixels,sigmaExpected_x__pixels,sigmaExpected_y__pixels])
        
    #last parameter in the fit_multiple_gaussians is similar to min_distance above, we should give half of the 
    #value here of min_distance   
    gaussians, gaussians_popt = fit_multiple_gaussians(single_frame_input,maximas,sigmas_guesses,5)
        
    accumulator = []
    for gaussian in gaussians:

        if(gaussian!=-1):
            amplitude = gaussian[0]

            #print(gaussian)
            mu_x     = int(gaussian[1][1]) ##this is going to be mu_z, previous code [1][0]
            mu_y     = int(gaussian[1][2]) ##need to finalise what this is (x or y) [1][1]
            mu_z     = int(gaussian[1][0]) ##need to finalise what this is (x or y) [1][2]
            ##sigmas will also change due to the above 
            sigma_x  = int(gaussian[2][1]) 
            sigma_y  = int(gaussian[2][2])
            sigma_z  = int(gaussian[2][0])
            accumulator.append(np.array([amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z]))
            
    accumulator = np.array(accumulator)
    print(accumulator.shape)
    df = pd.DataFrame()
    df['amplitude'] = accumulator[:,0]
    df['mu_x'] = accumulator[:,1]
    df['mu_y'] = accumulator[:,2]
    df['mu_z'] = accumulator[:,3]
    df['sigma_x'] = accumulator[:,4]
    df['sigma_y'] = accumulator[:,5]
    df['sigma_z'] = accumulator[:,6]
    df.head()
    
    error_list, index_list = check_fitting_error(single_frame_input,maximas,gaussians,sigmas_guesses)
    
    # Construct the filename
    '''
    filename_csv = f'df_c2_t{frame}.csv'
    file_path_csv = os.path.join(csv_save_dir, filename_csv)
    df.to_csv(file_path_csv)
    '''

    # Construct the filename based on the loop index (time_frame)
    filename_pkl = f'df_c3_t{frame}.pkl'

    # Construct the full file path by joining the directory and filename
    file_path = os.path.join(save_dir, filename_pkl)

    # Save the DataFrame to a pickle file with the specified path
    df.to_pickle(file_path)
end_time = time.time()

print('time taken (seconds)', end_time - start_time)